In [15]:
from sklearn.metrics.pairwise import cosine_similarity
import re
from pyjarowinkler import distance
from sklearn.feature_extraction.text import TfidfVectorizer
import itertools
from itertools import islice
import mysql.connector as sql
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.preprocessing import minmax_scale
import pandas as pd
import re
from numpy import NaN
import numpy as np

In [2]:
#Database connections

db_connection = sql.connect(host='54.189.68.146', database='grmds054_drup881', user='grmds_analyst', password='Cmethods1')

similarity_table = pd.read_sql('SELECT * FROM dr_recom_simliar_users', con=db_connection)

webinar_table = pd.read_sql('SELECT * FROM dr_webform_submission', con=db_connection)

user_table = pd.read_sql('SELECT * FROM dr_rmds_users', con=db_connection)

In [3]:

df=pd.read_excel('MyEvents+IM Data_clean.xlsx')

In [4]:
features=['Industry','organization_type']
for i in features:
    df = pd.concat([df, pd.get_dummies(df[i],drop_first=True,prefix=i)], axis=1)
    df = df.drop(i,axis=1)
    
    
a=len(df['Difficulty-Level'].unique())

dictionary={'Fresh noob to Beginner':1/a,'Beginner':2/a,'Beginner to Intermediate':3/a, 'Intermediate':4/a,
         'Intermediate to Advanced':5/a,'Advanced':6/a}

df['Difficulty-Level']=df['Difficulty-Level'].map(dictionary)

scaler1 = MinMaxScaler()
df[['Ticket Sold_norm','Ticket Available_norm']] = scaler1.fit_transform(df[['Ticket Sold','Ticket Available']])

In [5]:
#Calculating similarity for the available features

class recom_webinar(object):
    def __init__(self):
        
        self.Industry_list=[
            'Industry_financials', 'Industry_fundamentals', 'Industry_health care',
       'Industry_information technology', 'Industry_utilities'
        ]
        
        self.organization_list=[
            'organization_type_I.T ', 'organization_type_RMDS',
       'organization_type_community', 'organization_type_covid19',
       'organization_type_data', 'organization_type_financial',
       'organization_type_media and entertainment',
       'organization_type_medical', 'organization_type_university'
        ]
        
        
        
    def compare(self, info1, info2):
        
        results = {}
        
        #webinar in same organization
        results['Organization_sim'] = self.simple_text(info1['Organization'], info2['Organization'])
        
        
        #title similarity
        results['Title_sim'] = self.simple_text(info1['Title'], info2['Title'])
        
        # interest sim
        
        results['difficulty_list_sim'] = self.ordinal_similarity(info1['Difficulty-Level'], info2['Difficulty-Level'])
        
        results['Industry_list_sim'] = self.cos_sim([info1[x] if info1[x] != None else 0 for x in self.Industry_list], 
                                               [info2[x] if info2[x] != None else 0 for x in self.Industry_list])
        
        
        results['Organization_list_sim'] = self.cos_sim([info1[x] if info1[x] != None else 0 for x in self.organization_list], 
                                               [info2[x] if info2[x] != None else 0 for x in self.organization_list])
        
       
    
        results['Coding_sim'] = 1.0 if (info1['Coding language'] == info2['Coding language']) or (info1['Coding language'] is NaN and info2['Coding language'] is NaN) else 0
        
        results['Type_sim'] = 1.0 if info1['Type'] == info2['Type'] else 0
        
        
        
        
        l1=info1['Instructor'].split(',')
        l2=info2['Instructor'].split(',')

        l1_set=set(l1)
        l2_set=set(l2)
        
        val=len(l1_set)
        
        results['Instructor_sim'] = len(l1_set.intersection(l2_set))/val   
        
        
        
        
        # Event sim
        lan_list1 = [info1['Event']]
        lan_list2 = [info2['Event']]
        if len(lan_list1) == 0 or len(lan_list2) == 0:
            results['Event_sim'] = None
        else:
            results['Event_sim'] = self.long_text(lan_list1, lan_list2)
            
        return results
    
    def ordinal_similarity(self,u1,u2):
        return (1-abs(u1-u2))
        
        
    def simple_text(self,u1,u2):
        if u1 in [None,''] or u2 in [None,'']:
            return None
        u1 = re.sub(r'[^\w]', '', u1).lower()
        u2 = re.sub(r'[^\w]', '', u2).lower()
        return distance.get_jaro_distance(u1, u2)   #replaced function here
    
    def cos_sim(self,u1,u2):
        if u1==u2:
            return 1.0
        else:
            return cosine_similarity([u1], [u2])[0][0]
    
    def long_text(self,doc1,doc2):
        vectorizer = TfidfVectorizer()
        vectors = vectorizer.fit_transform(doc1+doc2)
        dense = vectors.todense()
        dense1 = dense[:len(doc1)]
        dense2 = dense[len(doc1):]
        pairs = list(itertools.product(dense1, dense2))
        return sum(map(lambda x : cosine_similarity(x[0], x[1])[0][0] , pairs)) / len(pairs)

In [6]:
#webinar similarity matrix

final_list=[]
recom = recom_webinar()
for i in range(1,87):
    pflist=[]
    for j in range(1,87):
        v=recom.compare(df.loc[df['id'] == i].to_dict('records')[0], df.loc[df['id'] == j].to_dict('records')[0])
        f=sum(v.values())
        f=f/9
        pflist.append(f)
        
    
    final_list.append(pflist)
    

final=np.array(final_list)
final_df = pd.DataFrame(final)

scaler = StandardScaler()
scaler.fit(final_df)
f1=scaler.transform(final_df)
final_standard = pd.DataFrame(f1)

scaler1 = MinMaxScaler()
scaler1.fit(final_standard)
f2=scaler1.transform(final_standard)
final_df = pd.DataFrame(f2)

In [12]:
#webinar recommendations for the user

def userwebinar(u):
    dict1={}
    list1=list(set(webinar_table[webinar_table['uid']==u]['webform_id']))
    
    for i in list1:

        c=df[df['webform_id']==i]['id']

        if len(c)>0:

            k=int(str(c).split()[1])

            idx=final_df[k].nlargest(2).idxmin()

            val=final_df[k].nlargest(2).min()

            dict1[idx]=val

    list_sim_user=similarity_table[(similarity_table['user']==u)]['sim_user'].tolist()

    for j in list_sim_user:

        list2=[]

        list2=list(set(webinar_table[webinar_table['uid']==j]['webform_id']))

        for k in list1:
            k1=df[df['webform_id']==k]['id']
            if len(k1)>0:
                k2=int(str(k1).split()[1])
                for l in list2:
                    l1=df[df['webform_id']==l]['id']
                    if len(l1)>0:
                        l2=int(str(l1).split()[1])
                        if final_df[k2][l2]>0:
                            dict1[l2]=final_df[k2][l2]
    
    if len(dict1)>0:
        data = list(dict1.items())
        an_array = np.array(data)
        a=an_array.shape[0]
        new_col = np.full((a, 1), u)
        all_data = np.hstack((new_col,an_array))

        column_values=['userid','webinarid','recom_score']
        df12 = pd.DataFrame(data = all_data, 
                      columns = column_values)

        convert_dict = {'userid': int,
                    'webinarid': int
                   }

        df12 = df12.astype(convert_dict)
    else:
        df12 = pd.DataFrame(columns = ['userid','webinarid','recom_score'])
        
        
    #popularity feature getting more weight     
    weblist=df12['webinarid'].to_list()

    for i in range(len(weblist)):
        t=df.loc[df['id'] == weblist[i]]['Ticket Sold'].tolist()[0]
        if t==0:
            pass
        else:
            tn=df.loc[df['id'] == weblist[i]]['Ticket Sold_norm'].tolist()[0] 
            tb=df12.loc[df12['webinarid']==weblist[i]]['recom_score'].tolist()[0]
            df12.loc[i,'recom_score']=tb+tn
            
    return df12

In [19]:
#user webinar recommender matrix

sid=webinar_table['sid'].to_list()
df_final=pd.DataFrame(columns=['userid', 'webinarid','recom_score'])
for i in sid:
    output=userwebinar(i)
    df_final=pd.concat([df_final,output])
df_final[['recom_score']] = minmax_scale(df_final[['recom_score']])

In [20]:
df_final

,userid,webinarid,recom_score
0,110,81,0.431787
1,110,2,0.472178
2,110,12,0.329232
3,110,22,0.061549
4,110,4,0.417605
...,...,...,...
10,2901,12,0.878127
11,2901,19,0.065979
12,2901,21,0.452398
13,2901,9,0.225412
